[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/romiaprilian7406/sp500-ratio-cluster/blob/main/notebooks/sp500_ratclust_dataset.ipynb)

# Import Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import time
import warnings
import os

# Global Configurations

In [2]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Ticker list Function

In [3]:
# Mengambil daftar ticker S&P 500 terbaru dari dataset publik
def get_sp500_tickers():
    try:
        url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'
        df = pd.read_csv(url)
        # Mengubah format ticker (misal: BRK.B -> BRK-B) agar sesuai dengan yfinance
        tickers = [t.replace('.', '-') for t in df['Symbol'].tolist()]
        print(f"Berhasil mendapatkan {len(tickers)} ticker S&P 500")
        return tickers
    except Exception as e:
        print(f"Gagal mengambil ticker: {e}")
        return []

# Data Fetching Function

In [4]:
def fetch_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        if not info or len(info) < 5: return None

        data = {
            'ticker': ticker,
            'sector': info.get('sector', 'Unknown'),
            'marketCap': info.get('marketCap'),

            # Valuation
            'trailingPE': info.get('trailingPE'),
            'forwardPE': info.get('forwardPE'),
            'priceToBook': info.get('priceToBook'),
            'enterpriseToRevenue': info.get('enterpriseToRevenue'),

            # Profitability
            'returnOnEquity': info.get('returnOnEquity'),
            'returnOnAssets': info.get('returnOnAssets'),
            'profitMargins': info.get('profitMargins'),
            'operatingMargins': info.get('operatingMargins'),

            # Solvency
            'debtToEquity': info.get('debtToEquity'),
            'currentRatio': info.get('currentRatio'),
            'quickRatio': info.get('quickRatio'),

            # Volatility & Growth
            'beta': info.get('beta'),
            'revenueGrowth': info.get('revenueGrowth'),
            'earningsGrowth': info.get('earningsGrowth'),
        }
        return data

    except:
        return None

# Simple Data Cleaning & Preprocessing

In [5]:
# Membersihkan dataset mentah dengan logika strict namun sesuai konteks Clustering, dengan penyesuaian kolom dan logika negatif
def clean_data(df):
    df = df.copy()

    # 1. Drop Duplicates
    if 'ticker' in df.columns:
        df = df.drop_duplicates(subset=['ticker'], keep='first')

    # 2. Type Casting (Memastikan semua angka terbaca sebagai Float)
    num_cols = [
        'marketCap', 'trailingPE', 'forwardPE', 'priceToBook',
        'enterpriseToRevenue', 'profitMargins', 'returnOnEquity', 'returnOnAssets',
        'operatingMargins', 'debtToEquity', 'currentRatio', 'quickRatio',
        'beta', 'revenueGrowth', 'earningsGrowth'
    ]

    # Filter hanya kolom yang benar-benar ada di DF
    cols_to_clean = [c for c in num_cols if c in df.columns]

    for col in cols_to_clean:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 3. Handle Infinite Values (Penting untuk K-Means)
    # Mengubah 'inf' (hasil pembagian nol) menjadi NaN agar bisa di-impute
    df = df.replace([np.inf, -np.inf], np.nan)

    # 4. Standardize Sector
    if 'sector' in df.columns:
        df['sector'] = df['sector'].fillna('Unknown').astype(str).str.strip().str.title()

    # 5. Strict Sanity Filter
    # Untuk Clustering, 'marketCap' adalah data wajib.
    # Rasio lain boleh NaN (karena akan  impute), tapi tanpa Market Cap, data tidak valid.
    subset_wajib = ['marketCap']
    # Pastikan kolom subset ada sebelum dropna
    valid_subset = [c for c in subset_wajib if c in df.columns]
    if valid_subset:
        df = df.dropna(subset=valid_subset)

    # 6. Logic Filter (Hanya Market Cap yang Wajib Positif)
    # TIDAK memfilter earningsGrowth/Margin negatif,
    # karena itu adalah fitur valid untuk cluster saham "Loss Making".
    if 'marketCap' in df.columns:
        df = df[df['marketCap'] > 0]

    # 7. Reorder Columns (Identity -> Fundamental)
    # Memastikan ticker dan sector ada di depan
    first_cols = ['ticker', 'sector', 'marketCap']
    other_cols = [c for c in df.columns if c not in first_cols]

    # Pastikan kolom yang diminta ada
    final_cols = [c for c in first_cols if c in df.columns] + other_cols
    df = df[final_cols]

    return df

# Main Execution

In [6]:
tickers = get_sp500_tickers()
data_list = []

for ticker in tqdm(tickers):
    stock_data = fetch_data(ticker)
    if stock_data:
        data_list.append(stock_data)
    time.sleep(0.5)

# Create Initial DataFrame
df_raw = pd.DataFrame(data_list)

Berhasil mendapatkan 503 ticker S&P 500


  0%|          | 0/503 [00:00<?, ?it/s]

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}


In [7]:
# Buat DataFrame
df_raw = pd.DataFrame(data_list)

if not df_raw.empty:
    # 1. Bersihkan Dataset (Fungsi ini butuh kolom 'ticker' lowercase)
    df = clean_data(df_raw)

    print(f"Total Raw Data  : {len(df_raw)}")
    print(f"Total Clean Data: {len(df)}")

    # Preview
    print("\nContoh Data:")
    display(df.head())
else:
    print("Gagal mengumpulkan data atau data kosong")
    df_final = pd.DataFrame()

Total Raw Data  : 502
Total Clean Data: 502

Contoh Data:


,ticker,sector,marketCap,trailingPE,forwardPE,priceToBook,enterpriseToRevenue,returnOnEquity,returnOnAssets,profitMargins,operatingMargins,debtToEquity,currentRatio,quickRatio,beta,revenueGrowth,earningsGrowth
0,MMM,Industrials,89204817920,26.753992,21.199999,19.224058,3.908,0.72921,0.07971,0.13700,0.24367,281.904,1.842,1.027,1.148,0.035,-0.375
1,AOS,Industrials,9533015040,18.336927,16.714987,5.135502,2.486,0.28209,0.13878,0.13851,0.18631,12.063,1.544,0.894,1.335,0.044,0.146
2,ABT,Healthcare,217696616448,15.713568,24.240310,4.268505,5.087,0.30620,0.06793,0.31880,0.19395,25.310,1.703,1.088,0.714,0.069,0.000
3,ABBV,Healthcare,399570305024,172.580150,18.638088,-151.224080,7.759,1.37961,0.09585,0.04004,0.35497,NaN,0.725,0.468,0.349,0.091,-0.887
4,ACN,Technology,166046285824,21.923520,18.947407,5.314263,2.340,0.25509,0.11183,0.11021,0.15220,25.380,1.420,1.301,1.261,0.073,-0.155


# Simple EDA

In [8]:
# Shape
rows, cols = df.shape
print(f"Dimensi Dataset")
print(f"Total Baris    : {rows}")
print(f"Total Kolom    : {cols}\n")

# Info
print(df.info(),"\n")

# Fitur adalah semua kolom
features_list = [col for col in df.columns]
print(f"Jumlah Fitur : {len(features_list)}")
print(f"Daftar Fitur : {features_list}\n")

# Statistik Deskriptif Fitur Numerik
print(f"\nStatistik Deskriptif")
# Filter hanya numerik
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

try:
    # Tampilkan transpose agar mudah dibaca
    display(df[numeric_cols].describe().T.round(2))
except:
    print(df[numeric_cols].describe().T.round(2))

# Missing Values
print(f"\nLaporan Missing Values:")
missing_count = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Jumlah Hilang': missing_count,
    'Persentase (%)': missing_pct
})

# Filter hanya yang ada missing value & Sort dari yang terbanyak
missing_only = missing_df[missing_df['Jumlah Hilang'] > 0].sort_values(by='Jumlah Hilang', ascending=False)

# Format persentase setelah sorting (supaya bisa di-sort secara numerik dulu)
missing_only['Persentase (%)'] = missing_only['Persentase (%)'].map('{:.2f}%'.format)

if not missing_only.empty:
    print(missing_only)
else:
    print("Tidak ada missing values")

Dimensi Dataset
Total Baris    : 502
Total Kolom    : 17

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ticker               502 non-null    object 
 1   sector               502 non-null    object 
 2   marketCap            502 non-null    int64  
 3   trailingPE           476 non-null    float64
 4   forwardPE            502 non-null    float64
 5   priceToBook          502 non-null    float64
 6   enterpriseToRevenue  498 non-null    float64
 7   returnOnEquity       473 non-null    float64
 8   returnOnAssets       502 non-null    float64
 9   profitMargins        502 non-null    float64
 10  operatingMargins     502 non-null    float64
 11  debtToEquity         448 non-null    float64
 12  currentRatio         484 non-null    float64
 13  quickRatio           484 non-null    float64
 14  beta                 500 non-nul

,count,mean,std,min,25%,50%,75%,max
marketCap,502.0,1.311020e+11,4.375042e+11,4.089366e+09,2.033027e+10,3.817346e+10,8.372959e+10,4.441137e+12
trailingPE,476.0,3.813000e+01,7.369000e+01,4.540000e+00,1.766000e+01,2.468000e+01,3.418000e+01,1.133500e+03
forwardPE,502.0,2.536000e+01,3.788000e+01,-2.006200e+02,1.344000e+01,1.984000e+01,2.708000e+01,5.038100e+02
priceToBook,502.0,1.400000e+00,5.368000e+01,-9.450500e+02,1.720000e+00,3.110000e+00,6.880000e+00,1.894700e+02
enterpriseToRevenue,498.0,5.200000e+00,6.320000e+00,-3.470000e+00,2.050000e+00,3.870000e+00,6.470000e+00,1.096200e+02
returnOnEquity,473.0,2.500000e-01,4.900000e-01,-2.070000e+00,9.000000e-02,1.500000e-01,2.900000e-01,5.640000e+00
returnOnAssets,502.0,7.000000e-02,6.000000e-02,-1.500000e-01,3.000000e-02,5.000000e-02,9.000000e-02,5.400000e-01
profitMargins,502.0,1.400000e-01,1.500000e-01,-1.400000e+00,7.000000e-02,1.300000e-01,2.100000e-01,7.100000e-01
operatingMargins,502.0,2.200000e-01,1.500000e-01,-8.900000e-01,1.200000e-01,2.000000e-01,3.100000e-01,9.800000e-01
debtToEquity,448.0,1.401500e+02,2.811500e+02,5.300000e-01,3.983000e+01,7.546000e+01,1.413500e+02,4.217210e+03



Laporan Missing Values:
                     Jumlah Hilang Persentase (%)
earningsGrowth                  59         11.75%
debtToEquity                    54         10.76%
returnOnEquity                  29          5.78%
trailingPE                      26          5.18%
currentRatio                    18          3.59%
quickRatio                      18          3.59%
enterpriseToRevenue              4          0.80%
beta                             2          0.40%
revenueGrowth                    2          0.40%


# Export Dataset

In [10]:
file_name = 'sp500_ratclust_dataset.csv'
df.to_csv(file_name, index=False)
print(f"File tersimpan: {file_name}")

File tersimpan: sp500_ratclust_dataset.csv
